Get difficult cases

In [1]:
from utils import *

In [2]:
data_paths

{'data': '../data',
 'labels': '../data/5. Graphs Data/labels-affected_importers.csv',
 'graphs_data': '../data/5. Graphs Data',
 'atlas': '../data/2. Atlas',
 'uncomtrade': '../data/1. UN Comtrade',
 'baci': '../data/3. BACI',
 'additional': '../data/4. Additional Data Sources',
 'results': './gnn/results/'}

In [ ]:
graph_results = {}

# For each graph typy
for graph_type in ["export", "total", "export-layered", "multi-graph-total", "multi-graph-export"]:
    print(graph_type)
    results = pd.DataFrame()

    if graph_type.startswith("multi-graph"):
        node_ids = list(pd.read_csv(f"{data_paths['graphs_data']}/{graph_type}/node_features-2022-{graph_type.removeprefix('multi-graph-')}.csv")["country_id"])
    else:
        node_ids = []

        for prod_code in [f"{x:02d}" for x in range(1, 100)]:
            if prod_code in ["77", "98", "99"]: # 77 & 98 don't exist. 99 is not present in BACI
                continue
            print(len(list(pd.read_csv(f"{data_paths['graphs_data']}/{graph_type}/node_features-2021-{prod_code}-{graph_type}.csv")["country_id"])))
            node_ids += list(pd.read_csv(f"{data_paths['graphs_data']}/{graph_type}/node_features-2021-{prod_code}-{graph_type}.csv")["country_id"])

    # For each seed
    for seed in range(1, 11):

        seed_results = pd.DataFrame()

        # For each GNN model
        for model in ["GCN", "GAT", "SAGE"]:
            _ = json.load(open(f"{data_paths['results']}/{model}/{graph_type}/{model}-{graph_type}-{seed}-predictions.json"))["predictions"]
            _ = pd.DataFrame(_)
            _.columns = [model]
            seed_results = pd.concat([seed_results, _], axis=1) # Concatenate one column per model

        # Add the seed to keep track
        seed_results["SEED"] = seed
        # Add the true label
        seed_results["LABEL"] = json.load(open(f"{data_paths['results']}/{model}/{graph_type}/{model}-{graph_type}-{seed}-predictions.json"))["labels"]
        print(seed_results)
        seed_results["node_id"] = node_ids
        results = pd.concat([results, seed_results])

    # Add the majority vote for ENSEMBLE
    results["ENSEMBLE"] = results[["GCN", "GAT", "SAGE"]].sum(axis=1)
    results["ENSEMBLE"] = results.ENSEMBLE.apply(lambda x: 1 if x > 1 else 0)
    # Check if correct
    results["ENSEMBLE"] = (results.ENSEMBLE == results.LABEL).astype(int)
    results["GCN"] = (results.GCN == results.LABEL).astype(int)
    results["GAT"] = (results.GAT == results.LABEL).astype(int)
    results["SAGE"] = (results.SAGE == results.LABEL).astype(int)
    #results.reset_index(inplace=True, drop=False, allow_duplicates=True, names="node_ix")
    results_avg = results.groupby("node_id").mean().reset_index()
    graph_results[graph_type] = results_avg

export
213
230
229
230
214
216
229
232
229
226
227
223
210
193
231
229
229
231
231
231
232
232
228
225
230
199
233
231
231
233
225
231
232
232
229
217
216
231
234
233
195
233
191
232
205
216
205
232
231
185
193
219
200
220
222
229
226
222
229
218
232
232
231
233
228
216
223
232
230
231
231
234
233
229
205
232
205
218
202
206
231
231
235
235
222
235
227
227
235
225
224
211
233
231
229
225
       GCN  GAT  SAGE  SEED  LABEL
0        0    0     0     1      0
1        0    0     0     1      0
2        0    1     0     1      0
3        0    0     0     1      0
4        0    0     0     1      0
...    ...  ...   ...   ...    ...
21436    1    1     1     1      0
21437    0    1     1     1      0
21438    0    0     0     1      0
21439    1    0     0     1      0
21440    1    1     0     1      0

[21441 rows x 5 columns]


ValueError: Length of values (21479) does not match length of index (21441)

In [ ]:
import pickle
G = pickle.load(open(f"{data_paths['graphs_data']}/export-layered/graph-2022-export-layered.pkl", "rb"))
    

In [45]:
len(json.load(open(f"{data_paths['results']}/GCN/export/GCN-export-{seed}-predictions.json"))["predictions"])

21441

In [35]:
graph_results["total"]

,node_ix,GCN,GAT,SAGE,SEED,LABEL,ENSEMBLE
0,0,1.0,1.0,1.0,5.5,0.0,1.0
1,1,0.9,1.0,1.0,5.5,0.0,1.0
2,2,0.0,0.2,0.7,5.5,0.0,0.2
3,3,1.0,1.0,1.0,5.5,0.0,1.0
4,4,1.0,1.0,1.0,5.5,0.0,1.0
...,...,...,...,...,...,...,...
21436,21436,0.0,0.0,0.1,5.5,0.0,0.0
21437,21437,1.0,0.5,1.0,5.5,0.0,1.0
21438,21438,0.6,0.5,1.0,5.5,0.0,0.8
21439,21439,0.1,1.0,1.0,5.5,0.0,1.0


In [ ]:
# For each graph typy
for graph_type in ["export", "total", "export-layered", "multi-graph-total", "multi-graph-export"]:

    results = pd.DataFrame()

    # For each seed
    for seed in range(1, 11):

        seed_results = pd.DataFrame()

        # For each GNN model
        for model in ["GCN", "GAT", "SAGE"]:
            _ = json.load(open(f"./results/{model}/{graph_type}/{model}-{graph_type}-{seed}-predictions.json"))["predictions"]
            _ = pd.DataFrame(_)
            _.columns = [model]
            seed_results = pd.concat([seed_results, _], axis=1) # Concatenate one column per model

        # Add the seed to keep track
        seed_results["SEED"] = seed
        # Add the true label
        seed_results["LABEL"] = json.load(open(f"./results/{model}/{graph_type}/{model}-{graph_type}-{seed}-predictions.json"))["labels"]
        results = pd.concat([results, seed_results])

    # Add the majority vote for ENSEMBLE
    results["ENSEMBLE"] = results[["GCN", "GAT", "SAGE"]].sum(axis=1)
    results["ENSEMBLE"] = results.ENSAMBLE.apply(lambda x: 1 if x > 1 else 0)

    # Calculate the result metrics for each seed
    for seed in range(1,11):

        results_seed = results[results.SEED == seed]

        acc = accuracy_score(results_seed["LABEL"], results_seed["ENSEMBLE"])
        f1_macro_avg = f1_score(results_seed["LABEL"], results_seed["ENSEMBLE"], average="macro")
        f1_pos = f1_score(results_seed["LABEL"], results_seed["ENSEMBLE"], pos_label=1, average="binary")

        d = classification_report(results_seed["LABEL"], results_seed["ENSEMBLE"], output_dict=True)
        
        d["Accuracy"] = acc
        d["F1 - Avg. Macro"] = f1_macro_avg
        d["F1 - Positives"] = f1_pos
        

        with open(f"./results/ENSAMBLE/{graph_type}/ENSAMBLE-{graph_type}-{seed}-report.json", "w") as file:
            json.dump(d, file, indent=4)